In [ ]:
import os
import joblib
import time
import statistics

n_parallel = 8
n_files = 100

def calc_scores():
    os.system("g++ -O2 -std=c++17 main.cpp -o exe")
    scores = joblib.Parallel(n_jobs=n_parallel) (
        joblib.delayed(calc_score_each)(i) for i in range(n_files)
    )
    return scores

def calc_score_each(seed: int):
    in_file = f"input/{seed:04}.txt"
    out_file = f"output/{seed:04}.txt"
    os.makedirs("tools/output", exist_ok=True)
    os.system(f"./exe < tools/{in_file} > tools/{out_file} 2> /dev/null")
    cmd = f"cd tools && dotnet tester.dll judge -i {in_file} -o {out_file} 2> /dev/null"
    out = os.popen(cmd)
    res = out.read().split()[-1]
    print('seed: {}, res: {}'.format(seed, res))
    return int(res)


In [ ]:
srt = time.time()
scores = calc_scores()
print("elapsed time: {} s".format(time.time() - srt))
print(statistics.mean(scores))

In [ ]:
import matplotlib.pyplot as plt
plt.hist(scores)
plt.show()

In [ ]:
data_x = [[] for _ in range(9)]
data_y = [[] for _ in range(9)]
with open('cluster.txt') as f:
    line = f.readline()
    while(line):
        c, x, y = map(int, line.split())
        data_x[c].append(x)
        data_y[c].append(y)
        line = f.readline()

for i in range(9):
    plt.scatter(data_x[i], data_y[i], label=str(i))
plt.legend(loc='best')
plt.show()

In [ ]:
import optuna

def calc_scores_op(l_ratio, r_ratio):
    os.system("g++ -O2 -std=c++17 -DOPTUNA main.cpp -o exe")
    scores = joblib.Parallel(n_jobs=n_parallel) (
        joblib.delayed(calc_score_each_op)(i, l_ratio, r_ratio) for i in range(n_files)
    )
    return scores

def calc_score_each_op(seed: int, l_ratio, r_ratio):
    in_file = f"input/{seed:04}.txt"
    out_file = f"output/{seed:04}.txt"
    os.makedirs("tools/output", exist_ok=True)
    os.system(f"./exe {l_ratio} {r_ratio} < tools/{in_file} > tools/{out_file} 2> /dev/null")
    cmd = f"cd tools && dotnet tester.dll judge -i {in_file} -o {out_file} 2> /dev/null"
    out = os.popen(cmd)
    res = out.read().split()[-1]
    #print('seed: {}, res: {}'.format(seed, res))
    return int(res)
    

def objective(trial: optuna.trial.Trial):
    start = time.time()
    l_ratio = trial.suggest_int("climb1", 1, 100)
    r_ratio = trial.suggest_int("climb2", 1, 100)
    scores = calc_scores_op(l_ratio, r_ratio)
    #print(f"elapsed: {time.time() - start}")
    return statistics.mean(scores)

if __name__ == "__main__":
    study = optuna.create_study(
        direction="maximize",
        storage="sqlite:///mtk005.db",
        study_name="tune_range",
        
        load_if_exists=True,
    )
    study.optimize(objective, n_trials=10000)